# Process one-group actinide cross section uncertainties for different libraries

In [ ]:
import sandy

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

In [ ]:
import pandas as pd

In [ ]:
import logging
logging.getLogger().setLevel(logging.WARN)

In [ ]:
nuclides = [922350, 922360, 922380, 932370, 932380, 942380, 942390, 942400, 942410, 942420, 952410, 952421, 952430, 962440]
libs = ["jeff_33", "jendl_40u", "endfb_80"]

In [ ]:
# Options
iwt = 5  # PWR spectrum
ek = [1e-5, 2e7]  # 1 group
mt = [18, 102]  # two reactions: fission and capture

In [ ]:
stds = []
for lib in libs:
    for zam in nuclides:
        nucl = sandy.zam.zam2nuclide(zam)
        print(f"processing nuclide {nucl} for library {lib.upper()}...")
        err = sandy.get_endf6_file(lib, "xs", zam).get_errorr(
            err=0.1, verbose=False, dryrun=False,
            xs=True, chi=False, nubar=False, mubar=False,
            errorr33_kws=dict(
                irespr=0,
                iwt=iwt,
                ek=ek,
                mt=mt,
            ),
        )
        if "errorr33" in err:
            std = err["errorr33"].get_cov().get_std().multiply(100).reset_index().assign(NUCL=nucl, LIB=lib)
            stds.append(std)
df = pd.concat(stds, ignore_index=True)

In [ ]:
print(df.query("MT in @mt").to_string())